In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

2025-01-21 12:33:55.081702: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-21 12:33:55.098851: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737443035.114101  262440 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737443035.118338  262440 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-21 12:33:55.135028: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [17]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np

# Min-max scale function
def min_max_scale_sample(X_train, X_test):
    min_vals = X_train.min(axis=0, keepdims=True)  # Calculate on training data
    max_vals = X_train.max(axis=0, keepdims=True)  # Calculate on training data
    
    # Scale training data
    X_train_scaled = (X_train - min_vals) / (max_vals - min_vals)
    
    # Scale test data using the same min and max values from training
    X_test_scaled = (X_test - min_vals) / (max_vals - min_vals)
    
    return X_train_scaled, X_test_scaled

# Load the data
df = pd.read_csv("left.csv")

# Prepare feature matrix (X) and labels (y)
# Only use the 42 landmarks, excluding 'Handedness' and 'Label'
X = df.drop(['Label', 'Handedness'], axis=1).values  # Exclude 'Handedness' and 'Label'
y = df['Label'].values

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42069)

# Normalize the data (min-max scaling)
X_train_scaled, X_test_scaled = min_max_scale_sample(X_train, X_test)

# Convert labels to integer and then to categorical (one-hot encoding)
unique_labels = np.unique(y)
label_to_int = {label: idx for idx, label in enumerate(unique_labels)}
y_int = np.array([label_to_int[label] for label in y])
y_categorical = to_categorical(y_int)

y_train_cat = to_categorical([label_to_int[label] for label in y_train])
y_test_cat = to_categorical([label_to_int[label] for label in y_test])

# Build the model
model = Sequential([
    Input(shape=(X_train_scaled.shape[1],)),
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(len(unique_labels), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train_cat, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test_cat)
print(f'Test accuracy: {test_accuracy:.4f}')


Epoch 1/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4164 - loss: 1.5748 - val_accuracy: 0.9167 - val_loss: 0.3643
Epoch 2/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9371 - loss: 0.2624 - val_accuracy: 0.9815 - val_loss: 0.0786
Epoch 3/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9740 - loss: 0.0944 - val_accuracy: 0.9954 - val_loss: 0.0345
Epoch 4/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9939 - loss: 0.0316 - val_accuracy: 0.9954 - val_loss: 0.0221
Epoch 5/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9958 - loss: 0.0165 - val_accuracy: 0.9907 - val_loss: 0.0185
Epoch 6/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9802 - loss: 0.0619 - val_accuracy: 0.9907 - val_loss: 0.0324
Epoch 7/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9954 - loss: 0.0196 - val_accuracy: 0.9907 - val_loss: 0.0266
Epoch 8/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9969 - loss: 0.0092 - val_accuracy: 1.0000 - v

In [18]:
X_train.shape

(2152, 42)

In [19]:
model.save('hand_model_left.h5')

print("Model saved to 'hand_model.h5'.")

Model saved to 'hand_model.h5'.
